In [28]:
import quimb.tensor as qtn
import numpy as np

In [29]:
N = 2 # Number of qubits
circ = qtn.Circuit(N)

In [30]:
def two_qubit_brickwork_state_init(qtm_circ):
    "A function to initialize a two qubit Brickwork-like state"
    # Hamadard on qubits on which to propagate the state 
    for i in [1]:
        qtm_circ.apply_gate('H', i)
        
    # Create the entanglement for the brickwork state
    qtm_circ.apply_gate('CZ', 0, 1)
    return qtm_circ


In [31]:
# Flip the state to propagate from the first qubit for a better demonstration
circ.apply_gate('X',0)

# Print the statevector to see the initial state
print(np.round(circ.psi.to_qarray(), 10))


[[0.+0.j]
 [0.+0.j]
 [1.+0.j]
 [0.+0.j]]


In [32]:
# Initiate brickwork state
circ = two_qubit_brickwork_state_init(circ)

In [33]:
# Print the state vector to understand the two qubit brickwork-like state
print(np.round(circ.psi.to_qarray(), 10))

[[ 0.      +0.j]
 [ 0.      +0.j]
 [ 0.707107+0.j]
 [-0.707107+0.j]]


In [34]:
# Build the projector on the plus state to deterministically "simulate" measurement of the plus state 
hadamard_zeroket_zerobra_hadamard = np.matmul(np.matmul(np.array([[1, 1], [1,-1]]), np.array([[1, 0], [0,0]])), np.array([[1, 1], [1,-1]]))
proj = 1/2 * hadamard_zeroket_zerobra_hadamard 

def rotation_z_gate(theta):
    rot_z = np.array([[np.exp(-1j*theta), 0],[0, np.exp(1j*theta)]])
    return rot_z

def z_rot_proj_meas(qubit, theta, qtm_circ):
    # Rotate qubit to be measured
    z_rot = rotation_z_gate(theta)

    # Apply the "measurement" projector
    qtm_circ.apply_gate(proj, qubit)

    # Reormalize the state
    state_vector = np.round(qtm_circ.psi.to_qarray(), 10)
    ## Compute renormalization factor from the norm of the vector after projective measurement
    renorm_factor = np.sqrt(np.dot(state_vector.conj().T, state_vector))
    ## Apply renormalization with identity matrix on measured qubit
    renorm_matrix = np.array([[1, 0], [0, 1]])* 1/renorm_factor
    qtm_circ.apply_gate(renorm_matrix, qubit)
    
    

In [35]:
# Measure the first qubit with a rotation angle of 0 to propagate its state to the second qubit
for i in [0]:
    z_rot_proj_meas(i, 0, circ)

In [36]:
# Apply Hadamard corrections
## To separate the state of the first qubit in the final state
circ.apply_gate('H', 0)
## To correct for the number of Hadamards in the protocol
circ.apply_gate('H', 1)

# Print indices of the nonzero elements in the final statevector to verify 
# that the initial state of the first qubit has propagated on the second
list = np.nonzero(np.round(circ.psi.to_qarray(), 10))
sv =np.round(circ.psi.to_qarray(), 10) #np.round(np.exp(-1j * np.pi/2)*circ.psi.to_qarray(), 10)
print(list[0], len(sv))

for i in list[0]:
    print(sv[i])


[[0.+0.j]
 [1.+0.j]
 [0.+0.j]
 [0.+0.j]]
